In [214]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed=42


import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [176]:
F = lambda x, y: x**2 - y
x = torch.tensor(1.)
y = F(x, torch.tensor(0.))
Hess = torch.tensor(torch.autograd.functional.hessian(F, (x, y)))
Hess

tensor([[2., 0.],
        [0., 0.]])

In [290]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
from gdeep.create_nets.sample_nn import SmoothNet

# create random network
torch.manual_seed(3)
n = 2 # dimensional input space
random_net = SmoothNet(0, [n,5,5,5])

# create grid and compute grid values
plot_range = 2.
steps = 50
x_range = torch.linspace(-plot_range, plot_range, steps)
y_range = torch.linspace(-plot_range, plot_range, steps)
grid_x, grid_y = torch.meshgrid(x_range, y_range)
mesh = torch.stack((grid_x, grid_y), dim=-1)
model_val = random_net(None, mesh.reshape((-1,2)))[:,0].detach().reshape((-1, grid_x.shape[-1]))

assert (model_val[0][0] == random_net(None, mesh[0][0]))[0].data.tolist()

# plot graph of random_net
assert (x_range.shape == y_range.shape),\
'x and y resolution have to be the same to\
avoid strange behaviour in Contour plot.'

trace_1 = go.Contour(
    z = model_val.transpose(0, 1),
    x = x_range,
    y = y_range,
    contours=dict(
            start=0.0,
            end=1.0,
            size=0.01,
            showlabels=True
        )
)

x = torch.tensor([-1., -1.7], requires_grad=True)

trace_2 = go.Scatter(x=x[:0].detach(), y=x[1:].detach()) 




f = lambda x: random_net(None, x)[0]
hessian = torch.autograd.functional.hessian(f, x)

eig_val, eig_vec = torch.symeig(hessian, eigenvectors=True)
gradient = torch.autograd.grad(f(x), x)[0]
normal_gradient = (gradient / torch.norm(gradient)).detach()
x.requires_grad = False



trace_3 = ff.create_quiver(
    [x[0]],
    [x[1]],
    [normal_gradient[0]],
    [normal_gradient[1]],
)

#fig = go.FigureWidget([trace_1, trace_2])
fig = go.FigureWidget([trace_1, trace_2, trace_3.data[0]])
fig['layout']['yaxis']['scaleanchor']='x'
fig.show()

In [291]:
projection_tangent = torch.ones(n) - torch.einsum('i,j->ij', gradient, gradient)
hessian = torch.autograd.functional.hessian(f, x)
curvature = (projection_tangent @ hessian @ projection_tangent)/torch.norm(gradient)
torch.symeig(curvature).eigenvalues

tensor([2.3842e-07, 1.1592e+01])